In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/RAG

/content/drive/MyDrive/RAG


In [ ]:
# ---------------------------------------------------------
# 🧠 STEP 1: JD ELIGIBILITY AGENT (LangGraph Node 1)
# Goal: Retrieve relevant job descriptions and extract key skills + importance
# ---------------------------------------------------------

# ✅ 1. Install Required Packages
!pip install -q langchain langgraph faiss-cpu huggingface-hub transformers sentence-transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883

In [ ]:
!pip install -q chardet


# **Eligibility_Checker_Agent**

In [ ]:


# ✅ 2. Import Libraries
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, END

from transformers import pipeline

In [ ]:
import os
import chardet
from langchain.schema import Document

jd_directory = "/content/drive/MyDrive/RAG/JD"
job_descriptions = []

for filename in os.listdir(jd_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(jd_directory, filename)
        # Step 1: Detect encoding
        with open(file_path, 'rb') as f:
            raw_data = f.read()
            result = chardet.detect(raw_data)
            detected_encoding = result['encoding'] or 'utf-8'

        try:
            with open(file_path, 'r', encoding=detected_encoding) as f:
                content = f.read()
                job_descriptions.append(Document(page_content=content, metadata={"source": filename}))
                print(f"[OK] Loaded {filename} with encoding: {detected_encoding}")
        except Exception as e:
            print(f"[ERROR] Skipping {filename} — could not read: {e}")

print(f"\n✅ Total JDs successfully loaded: {len(job_descriptions)}")


[OK] Loaded Senior Data Scientist.txt with encoding: Windows-1252
[OK] Loaded Principal Data Scientist - BFS.txt with encoding: Windows-1252
[OK] Loaded Senior ML Engineer (ML Ops).txt with encoding: Windows-1252
[OK] Loaded Senior  Lead AI.txt with encoding: Windows-1252
[OK] Loaded ML Engineer (ML Ops).txt with encoding: Windows-1252
[OK] Loaded Senior Lead NLPGen AI - Data and AI.txt with encoding: Windows-1252
[OK] Loaded Senior Data Scientist  Lead Data Scientist - NLP GenAI.txt with encoding: Windows-1252
[OK] Loaded Senior  Lead MLE - MLOps (DevOps).txt with encoding: Windows-1252
[OK] Loaded Senior AnalystData Scientist - NLPGenAI.txt with encoding: Windows-1252
[OK] Loaded Principal Architect - Data Modelling.txt with encoding: Windows-1252
[OK] Loaded Architect and Senior Architect - Data Governance.txt with encoding: ISO-8859-1
[OK] Loaded Senior Data Engineer - Data Modeling.txt with encoding: Windows-1252
[OK] Loaded Azure Architect.txt with encoding: Windows-1252

✅ Total

Custom JD Parser (Section-Aware)

Open AI

In [ ]:
import openai

openai.api_key = "<<API Key to be entered>>"
client = openai.OpenAI(api_key=openai.api_key)


In [ ]:
import openai
import os

openai.api_key = "<<API Key to be entered>>"
client = openai.OpenAI(api_key=openai.api_key)
# Load the key from environment variable

sample_jd = job_descriptions[0].page_content[:2000]

prompt = f"""
You are an intelligent HR assistant.
Given the following job description, Return the following fields in valid JSON format only.

1. Job Title
2. Summary
3. Responsibilities
4. Must-Have Skills
5. Nice-to-Have Skills
6. Work Environment
7. Compensation
8. About the Company

Only return the JSON object. Do NOT add markdown, explanations, or bullet points outside of the JSON.

Job Description:
{sample_jd}
"""

print("[INFO] Sending request to OpenAI API...")

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an HR assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    max_tokens=700
)

output_text = response.choices[0].message.content.strip()
print("[✅ OUTPUT RECEIVED]:\n")
print(output_text)


[INFO] Sending request to OpenAI API...
[✅ OUTPUT RECEIVED]:

```json
{
  "Job Title": "Senior / Lead Data Scientist",
  "Summary": "As a Senior Data Scientist, you will lead and contribute to developing sophisticated machine learning models, predictive analytics, and statistical analyses to solve complex business problems.",
  "Responsibilities": [
    "Lead and contribute to developing sophisticated machine learning models, predictive analytics, and statistical analyses.",
    "Demonstrate proficiency in programming languages such as Python or R.",
    "Write clean, efficient, and maintainable code.",
    "Use relevant libraries and frameworks like TensorFlow, PyTorch, and scikit-learn.",
    "Develop data-driven solutions and analyze complex datasets."
  ],
  "Must-Have Skills": [
    "Proficiency in Python or R",
    "Experience with TensorFlow, PyTorch, scikit-learn",
    "Strong problem-solving skills"
  ],
  "Nice-to-Have Skills": [],
  "Work Environment": "Chennai, Hyderabad, B

In [ ]:
import json
from langchain.schema import Document

def extract_sections_from_json(json_text, source_name="JD_1.txt"):
    # Clean out triple backticks and optional "json" label
    cleaned = json_text.strip()
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].strip()
    if cleaned.startswith("```"):
        cleaned = cleaned[len("```"):].strip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].strip()

    try:
        data = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("[❌ ERROR] JSON parsing failed:", e)
        print("[RAW TEXT]:\n", cleaned[:500])
        return []

    structured_docs = []
    for key, value in data.items():
        content = "\n".join(value) if isinstance(value, list) else str(value)
        if content.strip():
            structured_docs.append(
                Document(
                    page_content=content.strip(),
                    metadata={"source": source_name, "section": key}
                )
            )
    return structured_docs


In [ ]:
# structured_chunks = extract_sections_from_json(output_text, source_name="Senior Data Scientist.txt")
# print(f"[INFO] Structured Chunks Extracted: {len(structured_chunks)}\n")
# for doc in structured_chunks:
#     print(f"--- Section: {doc.metadata['section']} ---\n{doc.page_content[:300]}\n")


In [ ]:
import openai
import time

openai.api_key = "<<API Key to be entered>>"  # your actual key
client = openai.OpenAI(api_key=openai.api_key)

structured_chunks = []
max_retries = 2

for idx, doc in enumerate(job_descriptions):
    source = doc.metadata["source"]
    print(f"\n[INFO] Processing JD {idx + 1}/{len(job_descriptions)} → {source}")

    jd_text = doc.page_content[:2000]
    attempt = 0
    success = False

    while attempt <= max_retries and not success:
        attempt += 1
        print(f"[Attempt {attempt}] Processing...")

        prompt = f"""
You are an intelligent HR assistant.
Given the following job description, extract and return the following fields as JSON:

1. Job Title
2. Summary
3. Responsibilities
4. Must-Have Skills
5. Nice-to-Have Skills
6. Work Environment
7. Compensation
8. About the Company

Only return a valid JSON object. No explanation or extra formatting.

Job Description:
{jd_text}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an HR assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=700
            )

            response_text = response.choices[0].message.content.strip()

            docs = extract_sections_from_json(response_text, source_name=source)

            if docs:
                structured_chunks.extend(docs)
                print(f"[✅] Extracted {len(docs)} structured sections.")
                success = True
            else:
                print("[⚠️] Empty structured output. Retrying...")

        except Exception as e:
            print(f"[❌] Error on attempt {attempt} for {source} — {e}")

        time.sleep(2)

    if not success:
        print(f"[❌] Skipping {source} after {max_retries} failed attempts.")

print(f"\n[FINAL INFO] Total structured chunks created: {len(structured_chunks)}")



[INFO] Processing JD 1/13 → Senior Data Scientist.txt
[Attempt 1] Processing...
[✅] Extracted 7 structured sections.

[INFO] Processing JD 2/13 → Principal Data Scientist - BFS.txt
[Attempt 1] Processing...
[✅] Extracted 7 structured sections.

[INFO] Processing JD 3/13 → Senior ML Engineer (ML Ops).txt
[Attempt 1] Processing...
[✅] Extracted 7 structured sections.

[INFO] Processing JD 4/13 → Senior  Lead AI.txt
[Attempt 1] Processing...
[✅] Extracted 5 structured sections.

[INFO] Processing JD 5/13 → ML Engineer (ML Ops).txt
[Attempt 1] Processing...
[✅] Extracted 7 structured sections.

[INFO] Processing JD 6/13 → Senior Lead NLPGen AI - Data and AI.txt
[Attempt 1] Processing...
[✅] Extracted 5 structured sections.

[INFO] Processing JD 7/13 → Senior Data Scientist  Lead Data Scientist - NLP GenAI.txt
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing JD 8/13 → Senior  Lead MLE - MLOps (DevOps).txt
[Attempt 1] Processing...
[✅] Extracted 5 structured

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

# ✅ Step 1: Check how many chunks are available
print(f"[INFO] Total structured chunks available: {len(structured_chunks)}")

# ✅ Step 2: Load the embedding model (can take a few seconds)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("[INFO] Embedding model loaded: all-mpnet-base-v2")

# ✅ Step 3: Build FAISS index with progress tracking
print("[INFO] Generating embeddings and building FAISS index...")
vectorstore = FAISS.from_documents(list(tqdm(structured_chunks, desc="Embedding Chunks")), embedding_model)

# ✅ Step 4: Index built
print("[✅] FAISS Index created successfully from structured_chunks.")
print(f"[INFO] Total vectors in index: {vectorstore.index.ntotal}")




[INFO] Total structured chunks available: 81


/tmp/ipython-input-12-1647799290.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[INFO] Embedding model loaded: all-mpnet-base-v2
[INFO] Generating embeddings and building FAISS index...


Embedding Chunks: 100%|██████████| 81/81 [00:00<00:00, 607761.40it/s]


[✅] FAISS Index created successfully from structured_chunks.
[INFO] Total vectors in index: 81


In [ ]:
# vectorstore.save_local("jd_vector_index")
# print("[INFO] FAISS index saved to 'jd_vector_index/' directory.")


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = FAISS.load_local(
    "jd_vector_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

print(f"[INFO] Reloaded FAISS index with {vectorstore.index.ntotal} vectors.")


[INFO] Reloaded FAISS index with 71 vectors.


In [ ]:
import openai

client = openai.OpenAI(api_key="<<API Key to be entered>>")  # your actual key

response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-3.5-turbo"
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain RAG pipeline"}
    ],
    temperature=0.3,
    max_tokens=512
)

print(response.choices[0].message.content)


The RAG (Retrieval-Augmented Generation) pipeline is a method used in natural language processing (NLP) to improve the quality and relevance of generated text by combining retrieval-based and generation-based approaches. It is particularly useful for tasks that require generating informative and contextually appropriate responses or content, such as question answering, summarization, or dialogue systems.

Here's a breakdown of how the RAG pipeline works:

1. **Retrieval Component**:
   - The retrieval component is responsible for fetching relevant information from a large corpus or database. This is typically done using a retriever model, which can be based on techniques like dense passage retrieval (DPR) or traditional information retrieval methods like TF-IDF or BM25.
   - Given an input query, the retriever searches the corpus to find the most relevant documents or passages. These retrieved documents provide context and factual grounding for the subsequent generation step.

2. **Aug

In [ ]:
from openai import OpenAI
import json
import re


# ✅ 1. Initialize OpenAI client
client = OpenAI(api_key="<<API Key to be entered>>")

# ✅ 2. User query
user_query = "We are hiring a Data Analyst with focus on visualization and A/B testing"
print("\n[USER QUERY]:", user_query)

# ✅ 3. Retrieve top JD chunks
retrieved_docs = vectorstore.similarity_search(user_query, k=2)
print(f"\n[INFO] Retrieved Top {len(retrieved_docs)} Relevant JD Chunks")
for doc in retrieved_docs:
    print("--- Retrieved JD ---\n", doc.page_content[:300], "\nSource:", doc.metadata.get("source"))

# ✅ 4. Create prompt
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
skill_prompt = f"""
You are a recruitment assistant.
Given the following job description text, extract a list of key skills required and assign each a weight (percentage importance) between 0 and 100 based on relevance.

Job Description:
{retrieved_text}

Return the result as a list of JSON objects like:
[{{"skill": "Python", "importance": 95}}, ...]
"""

print("\n[INFO] Sending prompt to OpenAI API...\n")
print(skill_prompt[:500], "...")

# ✅ 5. Generate response
try:
    response = client.chat.completions.create(
        model="gpt-4o",  # or use "gpt-3.5-turbo" for lower cost
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": skill_prompt}
        ],
        temperature=0.3
    )

    result_text = response.choices[0].message.content
    print("\n[LLM RESPONSE]:\n", result_text)

    # ✅ Optional: parse JSON if returned properly
    json_block = re.search(r"```json(.*?)```", result_text, re.DOTALL)
    if json_block:
        json_str = json_block.group(1).strip()
        try:
            parsed = json.loads(json_str)
            print("\n[✅] Parsed Skills:\n")
            for skill in parsed:
                print(f"→ {skill['skill']}: {skill['importance']}%")
        except Exception as json_err:
            print(f"[⚠️] Failed to parse extracted JSON: {json_err}")
    else:
        print("[⚠️] No JSON block found in response.")

except Exception as e:
    print(f"[❌] Error during OpenAI generation: {e}")



[USER QUERY]: We are hiring a Data Analyst with focus on visualization and A/B testing

[INFO] Retrieved Top 2 Relevant JD Chunks
--- Retrieved JD ---
 Senior / Lead Data Scientist 
Source: Senior Data Scientist.txt
--- Retrieved JD ---
 We are looking for a Senior Analyst or Machine Learning Engineer who will work on a broad range of cutting-edge data analytics and machine learning problems across a variety of industries. 
Source: ML Engineer (ML Ops).txt

[INFO] Sending prompt to OpenAI API...


You are a recruitment assistant.
Given the following job description text, extract a list of key skills required and assign each a weight (percentage importance) between 0 and 100 based on relevance.

Job Description:
Senior / Lead Data Scientist

We are looking for a Senior Analyst or Machine Learning Engineer who will work on a broad range of cutting-edge data analytics and machine learning problems across a variety of industries.

Return the result as a list of JSON objects like:
[{"skill

# **Resume Matcher Agent**

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
import os

resume_folder = "/content/drive/MyDrive/RAG/Version 1"
resume_files = [os.path.join(resume_folder, f) for f in os.listdir(resume_folder) if f.endswith(".pdf")]

resume_docs = []
for path in resume_files:
    loader = PyPDFLoader(path)
    docs = loader.load()
    for doc in docs:
        doc.metadata["source"] = os.path.basename(path)  # Track which resume it came from
    resume_docs.extend(docs)

print(f"[INFO] Loaded {len(resume_docs)} total chunks from {len(resume_files)} resumes")


[INFO] Loaded 77 total chunks from 62 resumes


In [ ]:
import os
import openai
from langchain.document_loaders import PyPDFLoader

# Set API key
openai.api_key = "<<API Key to be entered>>"
client = openai.OpenAI(api_key=openai.api_key)

# Load resume (pick one for now)
resume_path = "/content/drive/MyDrive/RAG/Version 1/Abhishek_Anand_23BM6JP01_CV1.pdf"
loader = PyPDFLoader(resume_path)
resume_text = "\n".join([page.page_content for page in loader.load()])

# Define the prompt for section extraction
prompt = f"""
You are an intelligent resume parsing assistant.

Given the following resume text, extract the following sections and return a valid JSON object:
1. Full Name
2. Contact Information (phone, email, LinkedIn)
3. Professional Summary
4. Skills
5. Professional Experience
6. Education
7. Certifications
8. Projects or Achievements

Only return the JSON object. Do NOT add any explanation or markdown.

Resume:
{resume_text}
"""

# Send to OpenAI API
print("[INFO] Sending request to OpenAI API...")

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful resume parsing assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    max_tokens=5000
)

# Extract and print the response
output_text = response.choices[0].message.content.strip()
print("[✅ PARSED RESUME STRUCTURE]:\n")
print(output_text)


[INFO] Sending request to OpenAI API...
[✅ PARSED RESUME STRUCTURE]:

```json
{
  "Full Name": "Abhishek Anand",
  "Contact Information": {
    "phone": null,
    "email": null,
    "LinkedIn": null
  },
  "Professional Summary": null,
  "Skills": [
    "Python",
    "Communication",
    "SQL",
    "Statistical Data Analysis",
    "Excel",
    "Dashboarding",
    "Tableau",
    "Predictive Modeling"
  ],
  "Professional Experience": [
    {
      "Company": "Doubtnut",
      "Position": "Business Analyst",
      "Location": "Gurugram",
      "Duration": "Feb 2021 - May 2023",
      "Responsibilities": [
        "Maintained and analyzed key metrics across sales, user engagement, user profiles & retention parameters.",
        "Used SQL, Excel and Python for analytics and prepared live and interactive dashboards on Metabase.",
        "Tracked performances of various product features, campaigns & trends pertaining to the Doubtnut app.",
        "Conducted Root Cause and Ad hoc analyses r

In [ ]:
import json
from langchain.schema import Document

def extract_sections_from_json(json_text, source_name="JD_1.txt"):
    # Clean out triple backticks and optional "json" label
    cleaned = json_text.strip()
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].strip()
    if cleaned.startswith("```"):
        cleaned = cleaned[len("```"):].strip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].strip()

    try:
        data = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("[❌ ERROR] JSON parsing failed:", e)
        print("[RAW TEXT]:\n", cleaned[:500])
        return []

    structured_docs = []

    def flatten_value(value):
        if isinstance(value, list):
            if all(isinstance(item, dict) for item in value):
                return "\n\n".join(json.dumps(item, indent=2) for item in value)
            else:
                return "\n".join(str(item) for item in value)
        elif isinstance(value, dict):
            return json.dumps(value, indent=2)
        else:
            return str(value)

    for key, value in data.items():
        content = flatten_value(value)
        if content.strip():
            structured_docs.append(
                Document(
                    page_content=content.strip(),
                    metadata={"source": source_name, "section": key}
                )
            )
    return structured_docs


structured_chunks = extract_sections_from_json(output_text, source_name="Abhishek_Anand_23BM6JP01_CV1.pdf")
print(f"[INFO] Structured Chunks Extracted: {len(structured_chunks)}\n")
for doc in structured_chunks:
    print(f"--- Section: {doc.metadata['section']} ---\n{doc.page_content[:3000]}\n")


[INFO] Structured Chunks Extracted: 8

--- Section: Full Name ---
Abhishek Anand

--- Section: Contact Information ---
{
  "phone": null,
  "email": null,
  "LinkedIn": null
}

--- Section: Professional Summary ---
None

--- Section: Skills ---
Python
Communication
SQL
Statistical Data Analysis
Excel
Dashboarding
Tableau
Predictive Modeling

--- Section: Professional Experience ---
{
  "Company": "Doubtnut",
  "Position": "Business Analyst",
  "Location": "Gurugram",
  "Duration": "Feb 2021 - May 2023",
  "Responsibilities": [
    "Maintained and analyzed key metrics across sales, user engagement, user profiles & retention parameters.",
    "Used SQL, Excel and Python for analytics and prepared live and interactive dashboards on Metabase.",
    "Tracked performances of various product features, campaigns & trends pertaining to the Doubtnut app.",
    "Conducted Root Cause and Ad hoc analyses related to several crucial Key Performance Indicators (KPIs).",
    "Analyzed over 25 A/B exper

In [ ]:
import openai
import time

openai.api_key = "<<API Key to be entered>>"
client = openai.OpenAI(api_key=openai.api_key)

structured_chunks = []
max_retries = 2

for idx, doc in enumerate(resume_docs):
    source = doc.metadata["source"]
    print(f"\n[INFO] Processing Resume {idx + 1}/{len(resume_docs)} → {source}")

    resume_text = doc.page_content[:4000]  # adjusted to allow longer content
    attempt = 0
    success = False

    while attempt <= max_retries and not success:
        attempt += 1
        print(f"[Attempt {attempt}] Processing...")

        prompt = f"""
You are an intelligent HR assistant.
Given the following resume text, extract and return the following fields as JSON:

1. Full Name
2. Contact Information (phone, email, LinkedIn)
3. Professional Summary
4. Skills
5. Professional Experience
6. Education
7. Certifications
8. Projects or Achievements

Only return a valid JSON object. No explanation or extra formatting.

Resume Text:
{resume_text}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an HR assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=4000
            )

            response_text = response.choices[0].message.content.strip()
            docs = extract_sections_from_json(response_text, source_name=source)

            if docs:
                structured_chunks.extend(docs)
                print(f"[✅] Extracted {len(docs)} structured sections.")
                success = True
            else:
                print("[⚠️] Empty structured output. Retrying...")

        except Exception as e:
            print(f"[❌] Error on attempt {attempt} for {source} — {e}")

        time.sleep(2)

    if not success:
        print(f"[❌] Skipping {source} after {max_retries} failed attempts.")

print(f"\n[FINAL INFO] Total structured chunks created: {len(structured_chunks)}")



[INFO] Processing Resume 1/77 → Subhav_Chauhan_23BM6JP55_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing Resume 2/77 → Muskan_Agarwal_23BM6JP02_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing Resume 3/77 → Muskan_Agarwal_23BM6JP02_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 1 structured sections.

[INFO] Processing Resume 4/77 → Awadhesh_Mahto_23BM6JP15_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing Resume 5/77 → Awadhesh_Mahto_23BM6JP15_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 1 structured sections.

[INFO] Processing Resume 6/77 → N T_Ramakrishnan_23BM6JP28_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing Resume 7/77 → Sangita_Bhakat_23BM6JP49_CV1.pdf
[Attempt 1] Processing...
[✅] Extracted 8 structured sections.

[INFO] Processing Resume 8/77 → Sangita_Bhakat_23BM6JP49_CV1.pdf
[Attempt 1] Processing...
[✅]

In [ ]:
# ------------------------------------------------------
# Step 0: Imports and Initial Setup
# ------------------------------------------------------
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from tqdm import tqdm
import os

# Make sure structured_chunks is defined (from your previous extraction step)
assert len(structured_chunks) > 0, "No structured resume chunks found."

# ------------------------------------------------------
# Step 1: Load Embedding Model
# ------------------------------------------------------
print("[INFO] Loading embedding model...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("[INFO] Embedding model loaded: all-mpnet-base-v2")

# ------------------------------------------------------
# Step 2: Create FAISS Index
# ------------------------------------------------------
print("[INFO] Creating FAISS index from resume chunks...")
vectorstore = FAISS.from_documents(list(tqdm(structured_chunks, desc="Embedding Chunks")), embedding_model)
print("[✅] FAISS index created with", vectorstore.index.ntotal, "vectors")

# Save to disk
vectorstore.save_local("resume_faiss_index")
print("[INFO] Index saved to 'resume_faiss_index/'")

# ------------------------------------------------------
# Step 3: Load FAISS Index (for querying)
# ------------------------------------------------------
print("[INFO] Reloading FAISS index for matching agent...")
vectorstore = FAISS.load_local(
    "resume_faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
print(f"[✅] Reloaded FAISS index with {vectorstore.index.ntotal} vectors.")


[INFO] Loading embedding model...
[INFO] Embedding model loaded: all-mpnet-base-v2
[INFO] Creating FAISS index from resume chunks...


Embedding Chunks: 100%|██████████| 495/495 [00:00<00:00, 1248454.89it/s]


[✅] FAISS index created with 495 vectors
[INFO] Index saved to 'resume_faiss_index/'
[INFO] Reloading FAISS index for matching agent...
[✅] Reloaded FAISS index with 495 vectors.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<<API Key to be entered>>"


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Define embedding model
resume_embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load FAISS index for resumes
resume_vectorstore = FAISS.load_local(
    "resume_faiss_index",   # path to your saved index for resumes
    embeddings=resume_embedding_model,
    allow_dangerous_deserialization=True
)

print(f"[INFO] Resume Vectorstore loaded with {resume_vectorstore.index.ntotal} vectors")


[INFO] Resume Vectorstore loaded with 495 vectors


In [ ]:
job_description = "We are hiring a Data Analyst with focus on visualization and A/B testing"


In [ ]:
retrieved_resumes = resume_vectorstore.similarity_search(job_description, k=5)


In [ ]:
import openai
import json
import pandas as pd

openai.api_key = "<<API Key to be entered>>"
client = openai.OpenAI(api_key=openai.api_key)

top_resumes = retrieved_resumes
skill_requirements = [
    "Supply Chain Analytics",
    "Python Scripting",
    "Process Optimization",
    "Education from IIT/IIM/ISI"
]

results = []

for i, doc in enumerate(top_resumes, start=1):
    resume_text = doc.page_content
    source_file = doc.metadata["source"]

    system_prompt = "You are an expert resume evaluator who scores candidates on required skills from their resume."

    user_prompt = f"""
A company is hiring for a role with the following requirements:
{json.dumps(skill_requirements, indent=2)}

Score each requirement from 0 to 100 based on the resume content below.
Then compute the average as "Overall Match %" and add a 1-line comment.

Resume Source: {source_file}
Resume Content:
\"\"\"
{resume_text}
\"\"\"

Respond in JSON like:
{{
  "Name": "Extracted or inferred name",
  "Source": "{source_file}",
  "Supply Chain Analytics": ...,
  "Python Scripting": ...,
  "Process Optimization": ...,
  "IIT/IIM/ISI": ...,
  "Overall Match %": ...,
  "Comments": "..."
}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
        )


        # Extract JSON from the response
        text_response = response.choices[0].message.content
        json_start = text_response.find('{')
        json_end = text_response.rfind('}') + 1
        parsed = json.loads(text_response[json_start:json_end])
        results.append(parsed)

    except Exception as e:
        print(f"[❌ ERROR] Resume {source_file}: {e}")
        continue

# Convert to DataFrame
df = pd.DataFrame(results)
import pandas as pd

# Display top 5 in Colab
from IPython.display import display
display(df.head(5))


,Name,Source,Supply Chain Analytics,Python Scripting,Process Optimization,IIT/IIM/ISI,Overall Match %,Comments
0,Akshay K,Akshay_K_23BM6JP05_CV1.pdf,90,85,80,0,63.75,Strong experience in supply chain analytics an...
1,Dhananjay Kumar,Dhananjay_Kumar_23BM6JP016_CV1.pdf,0,80,0,0,20.00,The candidate has Python skills but lacks spec...
2,Soham Chatterjee,Soham_Chatterjee_23BM6JP52_CV1.pdf,30,0,50,0,20.00,The candidate has experience in data analysis ...
3,Himanshu Singh,Himanshu_Singh_23BM6JP18_CV1.pdf,0,90,0,0,22.50,The candidate has strong Python skills but lac...
4,Anuj,Anuj_23BM6JP10_CV1.pdf,0,70,0,0,17.50,The candidate has Python skills but lacks spec...


# **Link JD Agent → Resume Matcher Agent**

In [ ]:
from openai import OpenAI
import json
import re
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = FAISS.load_local(
    "jd_vector_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

print(f"[INFO] Reloaded FAISS index with {vectorstore.index.ntotal} vectors.")
print(f"[DEBUG] Embedding Dimension: {embedding_model.client.get_sentence_embedding_dimension()}")
print(f"[DEBUG] FAISS Index Dimension: {vectorstore.index.d}")



# ✅ 1. Initialize OpenAI client
client = OpenAI(api_key="<<API Key to be entered>>")

# ✅ 2. User query
user_query = "We are hiring a Data Analyst with focus on visualization and A/B testing"
print("\n[USER QUERY]:", user_query)



# ✅ 3. Retrieve top JD chunks
retrieved_docs = vectorstore.similarity_search(user_query, k=2)
print(f"\n[INFO] Retrieved Top {len(retrieved_docs)} Relevant JD Chunks")
for doc in retrieved_docs:
    print("--- Retrieved JD ---\n", doc.page_content[:300], "\nSource:", doc.metadata.get("source"))

# ✅ 4. Create prompt
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
skill_prompt = f"""
You are a recruitment assistant.
Given the following job description text, extract a list of key skills required and assign each a weight (percentage importance) between 0 and 100 based on relevance.

Job Description:
{retrieved_text}

Return the result as a list of JSON objects like:
[{{"skill": "Python", "importance": 95}}, ...]
"""

print("\n[INFO] Sending prompt to OpenAI API...\n")
print(skill_prompt[:500], "...")

# ✅ 5. Generate response
try:
    response = client.chat.completions.create(
        model="gpt-4o",  # or use "gpt-3.5-turbo" for lower cost
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": skill_prompt}
        ],
        temperature=0.3
    )

    result_text = response.choices[0].message.content
    print("\n[LLM RESPONSE]:\n", result_text)

    # ✅ Optional: parse JSON if returned properly
    json_block = re.search(r"```json(.*?)```", result_text, re.DOTALL)
    if json_block:
        json_str = json_block.group(1).strip()
        try:
            parsed = json.loads(json_str)
            skills_with_weights = {item['skill']: item['importance'] for item in parsed}
            print("\n[✅] Parsed Skills:\n")
            for skill in parsed:
                print(f"→ {skill['skill']}: {skill['importance']}%")
        except Exception as json_err:
            print(f"[⚠️] Failed to parse extracted JSON: {json_err}")
    else:
        print("[⚠️] No JSON block found in response.")

except Exception as e:
    print(f"[❌] Error during OpenAI generation: {e}")


[INFO] Reloaded FAISS index with 71 vectors.
[DEBUG] Embedding Dimension: 768
[DEBUG] FAISS Index Dimension: 768

[USER QUERY]: We are hiring a Data Analyst with focus on visualization and A/B testing

[INFO] Retrieved Top 2 Relevant JD Chunks
--- Retrieved JD ---
 Senior / Lead Data Scientist 
Source: Senior Data Scientist.txt
--- Retrieved JD ---
 We are looking for a Senior Analyst or Machine Learning Engineer who will work on a broad range of cutting-edge data analytics and machine learning problems across a variety of industries. 
Source: ML Engineer (ML Ops).txt

[INFO] Sending prompt to OpenAI API...


You are a recruitment assistant.
Given the following job description text, extract a list of key skills required and assign each a weight (percentage importance) between 0 and 100 based on relevance.

Job Description:
Senior / Lead Data Scientist

We are looking for a Senior Analyst or Machine Learning Engineer who will work on a broad range of cutting-edge data analytics and mach

In [ ]:
results = []

for i, doc in enumerate(top_resumes, start=1):
    resume_text = doc.page_content
    source_file = doc.metadata["source"]

    # Create skill score text block
    skill_text = "\n".join([f"- {skill} ({weight}%)" for skill, weight in skills_with_weights.items()])

    user_prompt = f"""
A company is hiring for a role with the following skill requirements and weights:
{skill_text}

Score each requirement from 0 to 100 based on the resume content below.
Then compute the **weighted average** as "Overall Match %" using the importance weights.

Resume Source: {source_file}

Resume Content:
\"\"\"{resume_text}\"\"\"

Respond in JSON like:
{{
  "Name": "Extracted or inferred name",
  "Source": "{source_file}",
  "Skill Scores": {{
    "Skill 1": ..., "Skill 2": ...
  }},
  "Overall Match %": ...,
  "Comments": "..."
}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful resume evaluator."},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3
        )

        output = response.choices[0].message.content
        json_start = output.find('{')
        json_end = output.rfind('}') + 1
        parsed = json.loads(output[json_start:json_end])
        results.append(parsed)

    except Exception as e:
        print(f"[❌] Failed for {source_file}: {e}")


In [ ]:
import pandas as pd
from IPython.display import display

df = pd.DataFrame(results)
display(df.head(10))  # Or save to CSV: df.to_csv("resume_scores.csv", index=False)


,Name,Source,Skill Scores,Overall Match %,Comments
0,Akshay K,Akshay_K_23BM6JP05_CV1.pdf,"{'Data Analysis': 85, 'Machine Learning': 20, ...",70.05,"Akshay has strong skills in data analysis, pro..."
1,Dhananjay Kumar,Dhananjay_Kumar_23BM6JP016_CV1.pdf,"{'Data Analysis': 50, 'Machine Learning': 80, ...",63.25,Dhananjay Kumar's resume highlights skills in ...
2,Soham Chatterjee,Soham_Chatterjee_23BM6JP52_CV1.pdf,"{'Data Analysis': 85, 'Machine Learning': 20, ...",63.50,Soham Chatterjee has strong skills in data ana...
3,Himanshu Singh,Himanshu_Singh_23BM6JP18_CV1.pdf,"{'Data Analysis': 60, 'Machine Learning': 70, ...",65.50,The resume demonstrates a strong proficiency i...
4,Anuj,Anuj_23BM6JP10_CV1.pdf,"{'Data Analysis': 60, 'Machine Learning': 0, '...",44.75,Anuj's resume highlights skills in Python and ...


Lang Graph

In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langgraph.graph import StateGraph
from typing import TypedDict, List
from langchain_core.documents import Document
import json, re, pandas as pd, fitz
from pathlib import Path

# ---------------------------
# 🔧 Setup
# ---------------------------
print("[INFO] Loading Embeddings and VectorStores...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

jd_vectorstore = FAISS.load_local("jd_vector_index", embeddings=embedding_model, allow_dangerous_deserialization=True)
resume_vectorstore = FAISS.load_local("resume_faiss_index", embeddings=embedding_model, allow_dangerous_deserialization=True)

print(f"[✅] JD Vectors: {jd_vectorstore.index.ntotal}, Resume Vectors: {resume_vectorstore.index.ntotal}")

client = OpenAI(api_key="<<API Key to be entered>>")  # Replace securely

# ---------------------------
# 🔎 JD Eligibility Agent
# ---------------------------
def jd_eligibility_agent(state):
    user_query = state["user_query"]
    print("\n[JD AGENT] Searching JD Vectorstore...")
    retrieved_docs = jd_vectorstore.similarity_search(user_query, k=2)
    print(f"[JD AGENT] Retrieved {len(retrieved_docs)} job descriptions")

    jd_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
    skill_prompt = f"""
    You are a recruitment assistant.
    Given the following job description text, extract a list of key skills required and assign each a weight (percentage importance) between 0 and 100.

    Job Description:
    {jd_text}

    Return as:
    [{{"skill": "Python", "importance": 95}}, ...]
    """

    print("[JD AGENT] Sending to LLM...")
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": skill_prompt}
        ],
        temperature=0.0, seed=42
    )
    result_text = response.choices[0].message.content
    json_block = re.search(r"```json(.*?)```", result_text, re.DOTALL)
    skills_with_weights = {}
    if json_block:
        parsed = json.loads(json_block.group(1).strip())
        skills_with_weights = {item["skill"]: item["importance"] for item in parsed}
    print(f"[JD AGENT] Extracted Skills: {skills_with_weights}\n")
    return {"skills": skills_with_weights, "retrieved_docs": retrieved_docs, "user_query": user_query}

# ---------------------------
# 📄 Resume Matcher Agent
# ---------------------------
def resume_matcher_agent(state):
    skills_with_weights = state["skills"]
    user_query = state["user_query"]

    print("[RESUME AGENT] Searching resume vectorstore...")
    top_resumes = resume_vectorstore.similarity_search(user_query, k=5)
    print(f"[RESUME AGENT] Retrieved {len(top_resumes)} resumes")

    results = []
    skill_text = "\n".join([f"- {skill} ({weight}%)" for skill, weight in skills_with_weights.items()])

    for doc in top_resumes:
        resume_text = doc.page_content
        source_file = doc.metadata.get("source", "Unknown")

        user_prompt = f'''
        A company is hiring for a role with the following skill requirements and weights:
        {skill_text}

        Score each requirement from 0 to 100 based on the resume content below.
        Then compute the **weighted average** as "Overall Match %" using the importance weights.

        Resume Source: {source_file}
        Resume Content:
        """{resume_text}"""

        Respond in JSON like:
        {{
          "Name": "Extracted or inferred name",
          "Source": "{source_file}",
          "Skill Scores": {{ "Skill 1": ..., "Skill 2": ... }},
          "Overall Match %": ..., "Comments": "..."
        }}
        '''

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful resume evaluator."},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0, seed=42
            )
            output = response.choices[0].message.content
            json_start = output.find('{')
            json_end = output.rfind('}') + 1
            parsed = json.loads(output[json_start:json_end])
            results.append(parsed)

        except Exception as e:
            print(f"[❌] Failed for {source_file}: {e}")

    df = pd.DataFrame(results)
    top5 = df.sort_values("Overall Match %", ascending=False).head(5)
    print("[RESUME AGENT] Top 5 Matches:")
    print(top5[["Name", "Source", "Overall Match %"]])
    return {"top_matches": top5, "skills": skills_with_weights}

# ---------------------------
# 🧠 LangGraph Definition
# ---------------------------
class ResumeMatchState(TypedDict):
    user_query: str
    retrieved_docs: List[Document]
    top_resumes: List[Document]
    skills: dict
    top_matches: pd.DataFrame

graph_builder = StateGraph(state_schema=ResumeMatchState)
graph_builder.add_node("JD_Eligibility", jd_eligibility_agent)
graph_builder.add_node("Resume_Matcher", resume_matcher_agent)
graph_builder.set_entry_point("JD_Eligibility")
graph_builder.add_edge("JD_Eligibility", "Resume_Matcher")
graph_builder.set_finish_point("Resume_Matcher")
graph = graph_builder.compile()

# ---------------------------
# 🔁 Run Pipeline
# ---------------------------
final_state = graph.invoke({
    "user_query": "We are hiring a Data Analyst with focus on visualization and A/B testing"
})

# Show results
print("\n[✅ FINAL MATCHES]")
display(final_state["top_matches"])
# Optionally save
# final_state["top_matches"].to_csv("top_resume_matches.csv", index=False)


[INFO] Loading Embeddings and VectorStores...
[✅] JD Vectors: 71, Resume Vectors: 495

[JD AGENT] Searching JD Vectorstore...
[JD AGENT] Retrieved 2 job descriptions
[JD AGENT] Sending to LLM...
[JD AGENT] Extracted Skills: {'Data Analysis': 90, 'Machine Learning': 95, 'Python': 85, 'Industry Knowledge': 80, 'Problem Solving': 85, 'Communication': 75}

[RESUME AGENT] Searching resume vectorstore...
[RESUME AGENT] Retrieved 5 resumes
[RESUME AGENT] Top 5 Matches:
               Name                              Source  Overall Match %
0          Akshay K          Akshay_K_23BM6JP05_CV1.pdf            70.25
3    Himanshu Singh    Himanshu_Singh_23BM6JP18_CV1.pdf            70.25
1   Dhananjay Kumar  Dhananjay_Kumar_23BM6JP016_CV1.pdf            66.25
2  Soham Chatterjee  Soham_Chatterjee_23BM6JP52_CV1.pdf            63.25
4              Anuj              Anuj_23BM6JP10_CV1.pdf            30.25

[✅ FINAL MATCHES]


,Name,Source,Skill Scores,Overall Match %,Comments
0,Akshay K,Akshay_K_23BM6JP05_CV1.pdf,"{'Data Analysis': 85, 'Machine Learning': 20, ...",70.25,"Akshay has strong experience in data analysis,..."
3,Himanshu Singh,Himanshu_Singh_23BM6JP18_CV1.pdf,"{'Data Analysis': 70, 'Machine Learning': 60, ...",70.25,"Himanshu has a strong foundation in Python, wh..."
1,Dhananjay Kumar,Dhananjay_Kumar_23BM6JP016_CV1.pdf,"{'Data Analysis': 50, 'Machine Learning': 100,...",66.25,Dhananjay Kumar's resume shows strong skills i...
2,Soham Chatterjee,Soham_Chatterjee_23BM6JP52_CV1.pdf,"{'Data Analysis': 85, 'Machine Learning': 20, ...",63.25,Soham Chatterjee has strong skills in data ana...
4,Anuj,Anuj_23BM6JP10_CV1.pdf,"{'Data Analysis': 50, 'Machine Learning': 0, '...",30.25,"The resume shows proficiency in Python, which ..."
